In [2]:
# Scrap up to 100 reviews for each {pension_name}
# Create the hotel's directory and save the 100 reviews to scrap_data/{pension_name}/reviews.txt
import os
import time
import random
from collections import Counter
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

class PensionReviewScraper:
    def __init__(self):
        self.browser = None

    def start_browser(self):
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        service = Service(executable_path=ChromeDriverManager().install())
        self.browser = webdriver.Chrome(service=service, options=options)

    def stop_browser(self):
        if self.browser:
            self.browser.quit()

    def scrap_reviews(self, url, max_reviews=100):
        reviews = []  # List to collect reviews

        self.browser.get(url)  # Load the page
        time.sleep(2)  # Wait for initial loading

        scroll_count = 0  # Scroll counter
        while len(reviews) < max_reviews and scroll_count < 10:
            # Scroll to the bottom of the page
            self.browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
            time.sleep(random.uniform(1, 2))   # Wait after scrolling
            
            # Extract review elements
            review_elements = self.browser.find_elements(By.CSS_SELECTOR, "#__next > section > div > div.css-1js0bc8 > div > div:nth-child(3) > div > div:nth-child(5) > div.css-1kpa3g > p")

            # Collect review text
            for element in review_elements:
                review = element.text
                if review:  # Exclude empty reviews
                    reviews.append(review)
                    if len(reviews) >= max_reviews:
                        break
            
            # Increment scroll count for the next scroll
            scroll_count += 1

        return reviews[:max_reviews]  # eturn only the maximum number of reviews
def save_reviews_to_txt(pension_name, reviews):
    dir_path = f"scrap_data/pension/{pension_name}"
    os.makedirs(dir_path, exist_ok=True)  # Create the pension directory
    file_path = os.path.join(dir_path, "reviews.txt")
    with open(file_path, "w", encoding="utf-8") as file:
        for i, review in enumerate(reviews, 1):
            file.write(f"{i}번째 리뷰:\n{review}\n\n")

def main():
    pension_url_lists =  [
        'https://www.yanolja.com/reviews/domestic/3020240',
        'https://www.yanolja.com/reviews/domestic/3009701',
        'https://www.yanolja.com/reviews/domestic/3004613',
        'https://www.yanolja.com/reviews/domestic/3016229',
        'https://www.yanolja.com/reviews/domestic/3003179',
        'https://www.yanolja.com/reviews/domestic/3017165',
        'https://www.yanolja.com/reviews/domestic/3016188',
        'https://www.yanolja.com/reviews/domestic/3006540',
        'https://www.yanolja.com/reviews/domestic/3003099',
        'https://www.yanolja.com/reviews/domestic/3017615'
]
    
    pension_name_lists = [
        '제주 써니펜션',
        '제주 바다위에펜션',
        '제주 나인스파빌펜션',
        '제주 빈티지1950펜션',
        '제주 푸르다오션펜션',
        '서귀포 모이라펜션',
        '서귀포 힐링스파펜션',
        '제주 제주에서돌고래를만난다면',
        '서귀포 마가렛펜션',
        '서귀포 산방산에머물다펜션'
    ]

    scraper = PensionReviewScraper()
    scraper.start_browser()

    try:
        for pension_name, pension_url in zip(pension_name_lists, pension_url_lists):
            reviews = scraper.scrap_reviews(pension_url)
            save_reviews_to_txt(pension_name, reviews)  # Save reviews to a text file
            print(f"Reviews for {pension_name} saved to scrap_data/pension/{pension_name}/reviews.txt")
    finally:
        scraper.stop_browser()

if __name__ == "__main__":
    main()


Reviews for 제주 써니펜션 saved to scrap_data/pension/제주 써니펜션/reviews.txt
Reviews for 제주 바다위에펜션 saved to scrap_data/pension/제주 바다위에펜션/reviews.txt
Reviews for 제주 나인스파빌펜션 saved to scrap_data/pension/제주 나인스파빌펜션/reviews.txt
Reviews for 제주 빈티지1950펜션 saved to scrap_data/pension/제주 빈티지1950펜션/reviews.txt
Reviews for 제주 푸르다오션펜션 saved to scrap_data/pension/제주 푸르다오션펜션/reviews.txt
Reviews for 서귀포 모이라펜션 saved to scrap_data/pension/서귀포 모이라펜션/reviews.txt
Reviews for 서귀포 힐링스파펜션 saved to scrap_data/pension/서귀포 힐링스파펜션/reviews.txt
Reviews for 제주 제주에서돌고래를만난다면 saved to scrap_data/pension/제주 제주에서돌고래를만난다면/reviews.txt
Reviews for 서귀포 마가렛펜션 saved to scrap_data/pension/서귀포 마가렛펜션/reviews.txt
Reviews for 서귀포 산방산에머물다펜션 saved to scrap_data/pension/서귀포 산방산에머물다펜션/reviews.txt


In [3]:
# Read scrap_data/{pension_name}/reviews.txt
# Output the frequency of nouns and verbs from the reviews of {pension_name}
# Save the results into scrap_data/{pension_name}/nouns_and_verbs.txt
from collections import Counter
from konlpy.tag import Okt
import os

def save_nouns_and_verbs_to_file(pension_name, nouns, verbs):
    dir_path = f"scrap_data/pension/{pension_name}"
    os.makedirs(dir_path, exist_ok=True)  # Create pension directory
    file_path = os.path.join(dir_path, "nouns_and_verbs.txt")
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(f"명사 빈도수:\n") # Noun Frequency
        for word, count in nouns:
            file.write(f"{word}: {count}\n")
        file.write(f"\n동사 빈도수:\n") # Verb Frequency
        for word, count in verbs:
            file.write(f"{word}: {count}\n")
        file.write("\n")

def main():
    pension_name_lists = [
        '제주 써니펜션',
        '제주 바다위에펜션',
        '제주 나인스파빌펜션',
        '제주 빈티지1950펜션',
        '제주 푸르다오션펜션',
        '서귀포 모이라펜션',
        '서귀포 힐링스파펜션',
        '제주 제주에서돌고래를만난다면',
        '서귀포 마가렛펜션',
        '서귀포 산방산에머물다펜션'
    ]

    try:
        for pension_name in pension_name_lists:
            okt = Okt()
            nouns = Counter()
            verbs = Counter()
            with open(f"scrap_data/pension/{pension_name}/reviews.txt", "r", encoding="utf-8") as file:
                reviews = file.readlines()
                for review in reviews:
                    pos = okt.pos(review.strip())
                    nouns.update([word for word, pos_tag in pos if pos_tag.startswith('Noun')])
                    verbs.update([word for word, pos_tag in pos if pos_tag.startswith('Verb')])
                nouns = nouns.most_common(50)
                verbs = verbs.most_common(50)
                print(f"Scraped Reviews for {pension_name}:")
                save_nouns_and_verbs_to_file(pension_name, nouns, verbs)
                print(f"  - Nouns and verbs saved for {pension_name}")
    except FileNotFoundError:
        print("Hotel review files not found.")
        return

if __name__ == "__main__":
    main()


Scraped Reviews for 제주 써니펜션:
  - Nouns and verbs saved for 제주 써니펜션
Scraped Reviews for 제주 바다위에펜션:
  - Nouns and verbs saved for 제주 바다위에펜션
Scraped Reviews for 제주 나인스파빌펜션:
  - Nouns and verbs saved for 제주 나인스파빌펜션
Scraped Reviews for 제주 빈티지1950펜션:
  - Nouns and verbs saved for 제주 빈티지1950펜션
Scraped Reviews for 제주 푸르다오션펜션:
  - Nouns and verbs saved for 제주 푸르다오션펜션
Scraped Reviews for 서귀포 모이라펜션:
  - Nouns and verbs saved for 서귀포 모이라펜션
Scraped Reviews for 서귀포 힐링스파펜션:
  - Nouns and verbs saved for 서귀포 힐링스파펜션
Scraped Reviews for 제주 제주에서돌고래를만난다면:
  - Nouns and verbs saved for 제주 제주에서돌고래를만난다면
Scraped Reviews for 서귀포 마가렛펜션:
  - Nouns and verbs saved for 서귀포 마가렛펜션
Scraped Reviews for 서귀포 산방산에머물다펜션:
  - Nouns and verbs saved for 서귀포 산방산에머물다펜션


In [5]:
# Create directories scrap_data/{pension_name}/nouns and scrap_data/{pension_name}/verbs
# and read noun and verb frequencies from scrap_data/{pension_name}/nouns_and_verbs.txt
# Then generate (noun frequency plot, wordcloud) and (verb frequency plot, wordcloud)
# and save them as scrap_data/{pension_name}/nouns/nouns_word_frequency_plot.png
# scrap_data/{pension_name}/nouns/nouns_wordcloud.png
# scrap_data/{pension_name}/verbs/verbs_word_frequency_plot.png
# scrap_data/{pension_name}/verbs/verbs_wordcloud.png
import os
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud
from konlpy.tag import Okt

def generate_word_frequency_plot_and_word_cloud(pension_name, nouns, verbs):
    # Create directories if they don't exist
    nouns_dir = f"scrap_data/pension/{pension_name}/nouns"
    verbs_dir = f"scrap_data/pension/{pension_name}/verbs"
    os.makedirs(nouns_dir, exist_ok=True)
    os.makedirs(verbs_dir, exist_ok=True)

    # Noun frequency plot
    if nouns:
        words_words, words_counts = zip(*nouns)
        plt.rcParams['font.family'] = 'malgun gothic'
        plt.rcParams['axes.unicode_minus'] = False
        plt.figure(figsize=(12, 6))
        plt.bar(words_words, words_counts, color='skyblue')
        plt.title(f'Word Frequency Plot for {pension_name} (Nouns)')
        plt.xlabel('Words')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'{nouns_dir}/{pension_name}_nouns_word_frequency_plot.png')
        plt.close()

        # Generate noun wordcloud
        wc = WordCloud(
            font_path='C:\\Windows\\Fonts\\malgunbd.ttf',
            max_words=100,
            background_color='white'
        )
        word_cloud_img = wc.generate_from_frequencies(dict(nouns))
        word_cloud_img.to_file(f'{nouns_dir}/{pension_name}_nouns_wordcloud.png')

    # Verb frequency plot
    if verbs:
        words_words, words_counts = zip(*verbs)
        plt.rcParams['font.family'] = 'malgun gothic'
        plt.rcParams['axes.unicode_minus'] = False
        plt.figure(figsize=(12, 6))
        plt.bar(words_words, words_counts, color='salmon')
        plt.title(f'Word Frequency Plot for {pension_name} (Verbs)')
        plt.xlabel('Words')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'{verbs_dir}/{pension_name}_verbs_word_frequency_plot.png')
        plt.close()

        # Generate verb wordcloud
        wc = WordCloud(
            font_path='C:\\Windows\\Fonts\\malgunbd.ttf',
            max_words=100,
            background_color='white'
        )
        word_cloud_img = wc.generate_from_frequencies(dict(verbs))
        word_cloud_img.to_file(f'{verbs_dir}/{pension_name}_verbs_wordcloud.png')

def main():
    pension_name_lists = [
        '제주 써니펜션',
        '제주 바다위에펜션',
        '제주 나인스파빌펜션',
        '제주 빈티지1950펜션',
        '제주 푸르다오션펜션',
        '서귀포 모이라펜션',
        '서귀포 힐링스파펜션',
        '제주 제주에서돌고래를만난다면',
        '서귀포 마가렛펜션',
        '서귀포 산방산에머물다펜션'
    ]

    for pension_name in pension_name_lists:
        nouns = []  # Initialize list for nouns
        verbs = []  # Initialize list for verbs

        # Read nouns and verbs from file
        with open(f"scrap_data/pension/{pension_name}/nouns_and_verbs.txt", "r", encoding="utf-8") as file:
            lines = file.readlines()

        # Extract nouns and verbs data
        found_nouns = False
        found_verbs = False
        for line in lines:
            line = line.strip()
            if line == "명사 빈도수:": # Noun Frequency
                found_nouns = True
                found_verbs = False
                continue
            elif line == "동사 빈도수:": # Verb Frequency
                found_nouns = False
                found_verbs = True
                continue
            elif line == "":
                found_nouns = False
                found_verbs = False
                continue
            if found_nouns:
                word, count = line.split(":")
                nouns.append((word.strip(), int(count.strip())))
            elif found_verbs:
                word, count = line.split(":")
                verbs.append((word.strip(), int(count.strip())))

        # Generate word frequency plot and word cloud
        generate_word_frequency_plot_and_word_cloud(pension_name, nouns, verbs)
        print(f"Word Frequency Plot and Word Cloud saved for {pension_name}")

if __name__ == "__main__":
    main()


Word Frequency Plot and Word Cloud saved for 제주 써니펜션
Word Frequency Plot and Word Cloud saved for 제주 바다위에펜션
Word Frequency Plot and Word Cloud saved for 제주 나인스파빌펜션
Word Frequency Plot and Word Cloud saved for 제주 빈티지1950펜션
Word Frequency Plot and Word Cloud saved for 제주 푸르다오션펜션
Word Frequency Plot and Word Cloud saved for 서귀포 모이라펜션
Word Frequency Plot and Word Cloud saved for 서귀포 힐링스파펜션
Word Frequency Plot and Word Cloud saved for 제주 제주에서돌고래를만난다면
Word Frequency Plot and Word Cloud saved for 서귀포 마가렛펜션
Word Frequency Plot and Word Cloud saved for 서귀포 산방산에머물다펜션
